loggers for pipeline

In [ ]:
#|default_exp logger

In [ ]:
%load_ext autoreload
%autoreload 2
from ipynb_path import *

In [ ]:
#| export
from __future__ import annotations
import relax
from relax.import_essentials import *
from relax.explain import prepare_cf_module, Explanation
from relax.methods import *
from relax.ml_model import *
from privacy.watermark import batched_watermark, keras2haiku, hard_kl_divergence, WatermarkConfig
from scipy.stats import ttest_rel
from pprint import pprint
import wandb
import einops
import jax.tree_util as jt
from plum import dispatch
from beartype import beartype as typecheck
from jaxtyping import Array, Key
import yaml
from pytz import timezone
from datetime import datetime


In [ ]:
#| export
def print_data(data: Dict):
    data = "\n".join([f"{k}: {v}" for k, v in data.items()])
    print(data)

In [ ]:
#| export
@typecheck
def unpack_losses(losses: Tuple[Optional[Array], ...]):
    assert len(losses) <= 3, "losses should be a tuple of length 3 at most"
    if len(losses) < 3:
        return losses + (None,) * (3 - len(losses))
    return losses

In [ ]:
#| export
def plot_loss(loss, title=None):
    import matplotlib.pyplot as plt

    l1, l2, l3 = unpack_losses(loss)
    fig, ax = plt.subplots()
    if l1 is not None:  ax.plot(l1, label='poison')
    if l2 is not None:  ax.plot(l2, label='validity')
    if l3 is not None:  ax.plot(l3, label='regularization')
    
    if title:
        ax.set_title(title)
    
    ax.legend()
    return fig

In [ ]:
#| export
def convert_exps_to_df(exp: Explanation, w_exp: Explanation):
    """Creates a DataFrame with features, types, and prediction outputs."""

    test_indices = jnp.array(exp.test_indices)
    B, C, K = exp.cfs[test_indices].shape

    data = np.concatenate([
        exp.xs[test_indices], 
        einops.rearrange(exp.cfs[test_indices], 'B C K -> (B C) K'), 
        einops.rearrange(w_exp.cfs[test_indices], 'B C K -> (B C) K'), 
    ])
    
    assert data.shape == (B * (1 + 2 * C), K)

    df = pd.DataFrame(data, columns=[f'feat_{i}' for i in range(K)])
    df['type'] = np.concatenate([
        np.full((B,), 'xs'), 
        np.full((B*C,), 'cfs'), 
        np.full((B*C,), 'w_cfs'), 
    ])
    df['output'] = exp.pred_fn(data).argmax(axis=1)

    return df

In [ ]:
#| export
def _log_data(data: dict[str, Any], path: Path):
    log_name = path / "logs.txt" # log scalar data here
    for k, v in data.items():
        k = k.replace(" ", "_") # replace space with underscore
        if isinstance(v, pd.DataFrame):
            v.to_csv(path / f"{k}.csv")
        else: # append k, v to the file
            with open(log_name, 'a') as f:
                f.write(f"{k}:{v}\n")

def _log_losses(losses: dict[tuple[Array, ...]], path: Path):
    for title, loss in losses.items():
        title = title.replace(" ", "_") # replace space with underscore
        # log losses
        l1, l2, l3 = unpack_losses(loss)
        pd.DataFrame({"poison": l1, "validity": l2, "regularization": l3}).to_csv(path / f"{title}.csv")
        # plot and save
        fig = plot_loss(loss, title)
        fig.savefig(path / f"{title}.png", dpi=300)

In [ ]:
#| export
class BaseLogger:
    def __init__(self, experiment_name: str = None, hparams: Dict = None):
        self.name = experiment_name or "experiment"

    def log(self, data: dict[str, Any], print: bool = True):        raise NotImplementedError

    def log_losses(self, losses: dict[str, tuple[Array, Array]]):   raise NotImplementedError

    def log_exps(self, exp: Explanation, w_exp: Explanation):       raise NotImplementedError


class PrintLogger(BaseLogger):
    def log(self, data: dict[str, Any], print: bool = True):
        if print: print_data(data)
    
    def log_losses(self, losses: dict[str, Tuple[Array, Array]]): pass

    def log_exps(self, exp: Explanation, w_exp: Explanation): pass

class Logger: 
    def __init__(self, experiment_name: str = None, hparams: Dict = None):
        self.name = experiment_name or "experiment"
        current_time_et = datetime.now().astimezone(timezone('US/Eastern')).strftime("%Y-%m-%d-%H:%M:%S")
        self.path = Path(f'logs/{self.name}/{current_time_et}')
        self.path.mkdir(parents=True, exist_ok=True)
        if hparams is not None:
            self.save_hparams(hparams)

    def save_hparams(self, hparams: Dict):  
        # save hparams as yaml file
        with open(self.path / "hparams.yaml", "w") as f:
            yaml.dump(hparams, f)

    def log(self, data: dict[str, Any], print: bool = True): 
        if print: print_data(data)
        _log_data(data, self.path)

    def log_losses(self, losses: Dict[str, Tuple[Array, Array]]):
        _log_losses(losses, self.path)

    def log_exps(self, exp: Explanation, w_exp: Explanation):
        df = convert_exps_to_df(exp, w_exp)
        df.to_csv(self.path / "data.csv")

In [ ]:
#| export
def vis_table(table: pd.DataFrame):
    return wandb.Table(dataframe=table)

def vis_series(arrays: List[Array]):
    arrays = jt.tree_map(lambda x: np.asarray(x), arrays)
    keys = ['Poison', 'Validity'] if len(arrays) == 2 else ['Poison', 'Validity', 'Regularization']
    return wandb.plot.line_series(
        xs=np.arange(arrays[0].shape[0]), ys=arrays, keys=keys
    )

def visualize(data: Any):
    if isinstance(data, pd.DataFrame):
        return vis_table(data)
    return data

In [ ]:
#| export
class WandbLogger(Logger):
    def __init__(
        self,
        project: str,
        user_name: str,
        experiment_name: str,
        hparams: dict
    ):
        self.run = wandb.init(
            project=project, entity=user_name, 
            name=experiment_name, config=hparams,
            settings=wandb.Settings(start_method="fork")
        )

    def log(self, data: Dict[str, Any], print: bool = True):
        if print: print_data(data)
        data = {k: visualize(v) for k, v in data.items()}
        self.run.log(data)

    def log_losses(self, losses: dict):
        for title, loss in losses.items():
            self.run.log({title: vis_series(loss)})

    def log_exps(self, exp: Explanation, w_exp: Explanation):
        test_indices = jnp.array(exp.test_indices)
        B, C, K = exp.cfs[test_indices].shape
        data = np.concatenate([
            exp.xs[test_indices], 
            einops.rearrange(exp.cfs[test_indices], 'B C K -> (B C) K'), 
            einops.rearrange(w_exp.cfs[test_indices], 'B C K -> (B C) K'), 
        ])
        
        assert data.shape == (B * (1 + 2 * C), K)
        df = pd.DataFrame(data, columns=[f'feat_{i}' for i in range(K)])
        df['type'] = np.concatenate([
            np.full((B,), 'xs'), np.full((B*C,), 'cfs'), np.full((B*C,), 'w_cfs'), 
        ])
        df['output'] = exp.pred_fn(data).argmax(axis=1)
        self.run.log({'data': vis_table(df)})
